# Import section
Import neccessary libraries for later use

In [ ]:
# Import libraries
import csv
import sys, os
import sqlite3
import calmap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta
from datetime import timedelta
from collections import deque
from scipy.stats import gaussian_kde

# Read data
Import data from the SQLite database.
At this point only the following data is read:

| dataframe | content |
|-----------|---------|
| data      | Daily summaries |
| activities|Trainingen|

In [ ]:
# Read data from the SQLite database
db_connection = sqlite3.connect('../data/fitbit.db')
data = pd.read_sql("select * from daily_summary",db_connection)
activities = pd.read_sql("select * from Training",db_connection)

# Data wrangling
Perform some generic data wrangling activities.
<BR/>
The following fields are added to the daily summary dataframe:
* Date, replaces original string version of the date
* Year, Month, Week, Weekday, Day
* YearMonth, YearWeek: combination of year and month or week number
* Awake Percentage, calculated percentage of awake time
* SLeep Start Hour, Wake Up Hour

In [ ]:
# Data wrangling

#Remove empty entrie
data = data[data['Steps'].notnull()]

# Convert data column to date type
data['Date'] = pd.to_datetime(data['Date'])
# Create weeknumber
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Week'] = data['Date'].dt.week
data['Weekday'] = data['Date'].dt.weekday
data['Day'] = data['Date'].dt.day
data['YearMonth'] = (data['Year'] % 100) * 100 + data['Month']
data['YearWeek'] = (data['Year'] % 100) * 100 + data['Week']

# Same for activities
activities['Date'] = pd.to_datetime(activities['Date'])
activities['Year'] = activities['Date'].dt.year
activities['Month'] = activities['Date'].dt.month
activities['Week'] = activities['Date'].dt.week
activities['Weekday'] = activities['Date'].dt.weekday
activities['Day'] = activities['Date'].dt.day
activities['YearMonth'] = (activities['Year'] % 100) * 100 + activities['Month']
activities['YearWeek'] = (activities['Year'] % 100) * 100 + activities['Week']


# Percentage of awake time to time in bed (related to efficiency)
data['Awake Percentage'] = data['Sleep Minutes Awake']/data['Time in Bed']*100 

# Sleep start hour. 
dt_format = "%Y-%m-%dT%H:%M:%S.%f"
data['Sleep Start Hour'] = data['Sleep Start Time'].map(lambda x: (datetime.strptime(str(x),dt_format)).hour+(datetime.strptime(str(x),dt_format)).minute/60.0, na_action = 'ignore')
# If the hour is after noon, then subtract 12 so that midnight becomes the 0 reference.

# Midnight is the baseline so that hours can be either + or - from midnight
ind = data[data['Sleep Start Hour'] > 12.0].index.tolist()
data.loc[ind,'Sleep Start Hour'] = data['Sleep Start Hour'].iloc[ind] - 24.0

#Waking up time
data['Wake Up Hour'] = data['Sleep Start Hour']+data['Time in Bed']/60

data.tail(5)

In [ ]:
st_mon = data['Steps'].groupby(data['YearMonth']).mean()

fig,axes = plt.subplots(figsize = (18,12), nrows = 2, ncols = 1)

plt.sca(axes[0])
_ = data.boxplot(column='Steps', by='YearMonth', ax =axes[0])
_ = plt.xticks(np.linspace(1,19,19))
_ = plt.title('Steps per month (sum)')

plt.sca(axes[1])
st_mon.plot(kind = 'bar', alpha = 0.5)
r = plt.title('Median step count over the months')

In [ ]:
st_week = data.groupby(['Year', 'Week'])['Steps'].mean()
st_week.plot(kind = 'bar', alpha = 0.5, figsize = (18,5))
r = plt.title('Median step count over the weeks')

In [ ]:
st_week_2017 = data[data['Year'] == 2017].groupby(['Week'])['Steps'].sum()
st_week_2018 = data[data['Year'] == 2018].groupby(['Week'])['Steps'].sum()
st_week_2019 = data[data['Year'] == 2019].groupby(['Week'])['Steps'].sum()

pd_2017 = pd.DataFrame(st_week_2017)
pd_2017.columns = ['2017']
pd_2018 = pd.DataFrame(st_week_2018)
pd_2018.columns = ['2018']
pd_2019 = pd.DataFrame(st_week_2019)
pd_2019.columns = ['2019']
ax = pd_2017.plot(figsize = (18,8))
pd_2018.plot(ax=ax)
pd_2019.plot(ax=ax)
ax.set_title('Steps per week')
r = ax.set_ylabel('Total steps')

In [ ]:
pd_2017_sum = pd_2017.cumsum()
pd_2018_sum = pd_2018.cumsum()
pd_2019_sum = pd_2019.cumsum()

ax = pd_2017_sum.plot(figsize = (18,8))
pd_2018_sum.plot(ax=ax)
pd_2019_sum.plot(ax=ax)
ax.set_title('Cumultive Steps over the year')
r = ax.set_ylabel('Total steps')

In [ ]:
# Delta 2019 tov 2018
delta = pd.DataFrame(pd_2019_sum)
delta['2018'] = pd.DataFrame(pd_2018_sum)
delta['Verschil'] = delta['2019'] - delta['2018']
dt = delta['Verschil']
axs = dt.plot(figsize = (18,6))
axs.set_ylim(-125000,0)
r = axs.set_title("Total diffence (cummulative) over the year")

In [ ]:
# Read step data per minute
steps_1min = pd.read_sql("select * from Steps_1m",db_connection)

In [ ]:
# create hourly figures
steps_1min['DateTime'] = pd.to_datetime(steps_1min['Date'] + ' ' + steps_1min['Time'], format='%Y-%m-%d %H:%M:%S')
steps_1min = pd.DataFrame(steps_1min)
steps_1min.index = pd.DatetimeIndex(steps_1min['DateTime'])
steps_1u = pd.DataFrame()
steps_1u['steps'] = steps_1min.Steps.resample('H').sum()

In [ ]:
steps_1u['Hour'] = steps_1u.index.hour
hour_steps = steps_1u.groupby([steps_1u.Hour]).mean()
plt.figure(figsize=(16,6))
plt.bar(hour_steps.index, hour_steps.steps)
plt.ylim(bottom=40)
plt.title("Steps per hour")
r = plt.xticks(hour_steps.index)

In [ ]:
# Heatmap steps per day
daily_steps = pd.DataFrame()
daily_steps['Date'] = data['Date']
daily_steps['Steps'] = data['Steps']

series = daily_steps
series['Date'] = pd.to_datetime(series['Date'], format='%Y-%m-%d')
series = series.set_index('Date')
series= pd.Series(series['Steps'], index = series.index)
r = calmap.calendarplot(series, monthticks=1, daylabels='MTWTFSS',
                    dayticks=1, linewidth=0,
                    fig_kws=dict(figsize=(16, 6)))

In [ ]:
# Steps per day of the week
weekday_steps = data.groupby(['Weekday']).mean()['Steps']
weekday_steps_tot = data.groupby(['Weekday']).sum()['Steps']
day_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

fig,axes = plt.subplots(figsize = (16,6),nrows = 1, ncols = 2)

plt.sca(axes[0])
ax = axes[0]
ax.bar(weekday_steps.index, weekday_steps)
plt.xticks(weekday_steps.index, day_labels)
plt.ylim(bottom=8000)
plt.title('Mean steps per day of the week')

plt.sca(axes[1])
ax = axes[1]
ax.bar(weekday_steps_tot.index, weekday_steps_tot)
plt.xticks(weekday_steps_tot.index, day_labels)
plt.ylim(bottom=700000)
r= plt.title('Total steps per day of the week')

In [ ]:
# Show days with most and least steps taken

largest = daily_steps.nlargest(5, 'Steps')
smallest = daily_steps[daily_steps['Steps'] > 1000]
smallest = smallest.nsmallest(5, 'Steps', keep='last')

fig,axes = plt.subplots(figsize = (16,6),nrows = 1, ncols = 2)
plt.sca(axes[0])
ax = axes[0]
ax.barh([1,2,3,4,5], largest.Steps)
dates = largest.Date.tolist()
dates = [datetime.strftime(i, "%Y-%m-%d") for i in dates]
plt.yticks([1,2,3,4,5], dates)
plt.xlim(left=10000, right=60000)
plt.title('Days with most steps')
plt.gca().invert_yaxis()
for i, v in enumerate(largest.Steps):
    ax.text(v, i + 1, str(v), color='blue', fontweight='bold')
    
plt.sca(axes[1])
ax = axes[1]
ax.barh([1,2,3,4,5], smallest.Steps)
dates = smallest.Date.tolist()
dates = [datetime.strftime(i, "%Y-%m-%d") for i in dates]
plt.yticks([1,2,3,4,5], dates)
plt.xlim(left=0, right=3500)
plt.title('Days with least steps')
plt.gca().invert_yaxis()
for i, v in enumerate(smallest.Steps):
    ax.text(v, i + 1, str(v), color='blue', fontweight='bold')


In [ ]:
# Read heart rate data per minute
hr_1min = pd.read_sql("select * from Heartrate",db_connection)
# create hourly figures
hr_1min['DateTime'] = pd.to_datetime(hr_1min['Date'] + ' ' + hr_1min['Time'], format='%Y-%m-%d %H:%M:%S')
hr_1min = pd.DataFrame(hr_1min)
hr_1min.index = pd.DatetimeIndex(hr_1min['DateTime'])
hr_1u = pd.DataFrame()
hr_1u['heartrate'] = hr_1min['Heart Rate'].resample('H').mean()
hr_1u['Hour'] = hr_1u.index.hour

In [ ]:
# Hourly heart rate (average) data
hour_hr = hr_1u.groupby([hr_1u.Hour]).mean()
plt.figure(figsize=(16,6))
plt.bar(hour_hr.index, hour_hr.heartrate)
plt.ylim(bottom=40)
plt.title("Heart rate per hour")
r = plt.xticks(hour_hr.index)

In [ ]:
hr_steps_1u = hr_1u.join(steps_1u, lsuffix='_hr', rsuffix='_steps')
hr_steps_1u = hr_steps_1u[hr_steps_1u['steps'] > 1000]

plt.figure(figsize=(16,6))
plt.scatter(hr_steps_1u.steps, hr_steps_1u.heartrate)
plt.title("Heartrate vs Steps (hourly)")
plt.xlabel('Steps')
plt.ylabel('Heart Rate')

In [ ]:
# Correlation heartrate and number of steps last 3 hours
# This correlation is less than actual steps per hour and heart rate
hs1u = hr_steps_1u['steps'].rolling(2).sum()
hs1u = pd.DataFrame(hs1u)
hs1u['heartrate'] = hr_steps_1u['heartrate']
plt.figure(figsize=(16,6))
plt.scatter(hs1u.steps, hr_steps_1u.heartrate)
plt.title("Heartrate vs Steps (3 hour sum)")
plt.xlabel('Steps')
plt.ylabel('Heart Rate')

In [ ]:
# Steps per day histogram
fig, ax = plt.subplots(figsize=(16,6))

daily_steps.hist(column=["Steps"], grid=False, bins=100, figsize = (18,6), ax=ax)
plt.xlabel('Steps')
plt.ylabel('Frequency')
_ = plt.title('Steps per day histogram')

In [ ]:
# Histogram per day of the week
daily_steps['Weekday'] = daily_steps['Date'].dt.weekday
weekdays = ['Mon','Tue','Wed','Thur','Fri','Sat','Sun']
daily_steps_per_day = []
fig, ax = plt.subplots(figsize=(16,6))
for x in range(7):
    steps = daily_steps[daily_steps['Weekday'] == x]
    density = gaussian_kde(steps['Steps'])
    xs = np.linspace(0,60000,60000)
    density.covariance_factor = lambda : .25
    density._compute_covariance()
    ax.plot(density(xs), label=weekdays[x])
plt.xlabel('Steps')
plt.ylabel('Frequency')
plt.legend()
r = plt.title('Steps per day of week histogram')

In [ ]:
st_week = pd.DataFrame(data.groupby(['YearWeek'])['Steps'].sum().tail(20))
x = [(i-len(st_week)+1) for i in range(len(st_week))]
plt.figure(figsize=(16,6))
plt.bar(x,st_week.Steps)
plt.ylim(bottom=0)
plt.title("Steps per week (last 20 weeks)")
plt.xlabel("Week")
plt.ylabel("Steps")
r = plt.xticks(x)

In [ ]:
st_week['dSteps'] = st_week['Steps'] - st_week['Steps'].shift(1)

In [ ]:
st_week.tail(10)

In [ ]:
st_week_2019 = st_week[st_week.index>1900]

plt.figure(figsize=(16,6))
plt.bar(st_week_2019.index, st_week_2019.dSteps)
plt.ylim(bottom=40)
plt.title("Delta number of steps compared to last week")
plt.ylim(bottom=-20000, top=20000)
plt.axhline(y=0)
plt.axhline(y=10000, linestyle=':')
plt.axhline(y=-10000, linestyle=':')
_ = plt.xticks(st_week_2019.index)

In [ ]:
MIN_STEPS_TRAINING = 5000
wandelingen = activities[activities['Steps'] >= MIN_STEPS_TRAINING]

In [ ]:
trainafstand_2017 = wandelingen[wandelingen['Year'] == 2017].groupby(['Week'])['Steps'].sum()
trainafstand_2018 = wandelingen[wandelingen['Year'] == 2018].groupby(['Week'])['Steps'].sum()
trainafstand_2019 = wandelingen[wandelingen['Year'] == 2019].groupby(['Week'])['Steps'].sum()

ta_2017 = pd.DataFrame(trainafstand_2017)
ta_2017.columns = ['2017']
ta_2018 = pd.DataFrame(trainafstand_2018)
ta_2018.columns = ['2018']
ta_2019 = pd.DataFrame(trainafstand_2019)
ta_2019.columns = ['2019']
ax = ta_2017.plot(figsize = (18,8))
ta_2018.plot(ax=ax)
ta_2019.plot(ax=ax)
ax.set_title('Training steps per week (min ' + str(MIN_STEPS_TRAINING) + ' steps / activity)')
r = ax.set_ylabel('Total steps')

In [ ]:
ta_2017_sum = ta_2017.cumsum()
ta_2018_sum = ta_2018.cumsum()
ta_2019_sum = ta_2019.cumsum()

ax = ta_2017_sum.plot(figsize = (18,8))
ta_2018_sum.plot(ax=ax)
ta_2019_sum.plot(ax=ax)
ax.set_title('Cumultive Trainimg Steps over the year')
r = ax.set_ylabel('Total steps')

In [ ]:
# Delta 2019 tov 2018
delta = pd.DataFrame(ta_2019_sum)
delta['2018'] = pd.DataFrame(ta_2018_sum)
delta['Verschil'] = delta['2019'] - delta['2018']
dt = delta['Verschil']

fig, ax = plt.subplots(figsize=(18,6))
g = sns.lineplot(data=dt, ax=ax)
r = ax.set_title("Total diffence (cummulative) over the year")
_ = plt.xlabel('Week')
_ = plt.ylabel('Steps')
_ = plt.axhline(y=0, linestyle=':')
plt.show()